## Tarea 16 del curso CM-072

* Nombre y apellidos:
* Fecha de presentación: 13 de diciembre.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

### Extracción de características: vectorizador

Para algunos tipos de datos, por ejemplo, datos de texto, se debe aplicar un paso de extracción de características para convertirlos en características numéricas. Para ilustrar esto usamos el conjunto de datos de spam de SMS.


In [2]:
#Modifica el path
import os
# print(os.getcwd())
with open(os.path.join("Tarea16", "SMSSpamCollection")) as f:
    lineas = [linea.strip().split("\t") for linea in f.readlines()]
texto = [x[1] for x in lineas]
y = [x[0] == "ham" for x in lineas]


In [3]:
from sklearn.model_selection import train_test_split
texto_entrenamiento, texto_prueba, y_entrenamiento, y_prueba = train_test_split(texto, y)


Aplicamos la extracción de características manualmente, así:

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer()
vectorizer.fit(texto_entrenamiento)

X_entrenamiento = vectorizer.transform(texto_entrenamiento)
X_prueba = vectorizer.transform(texto_prueba)

clf = LogisticRegression()
clf.fit(X_entrenamiento, y_entrenamiento)
clf.score(X_prueba, y_prueba)


0.9641319942611191

La situación en la que aprendemos una transformación y luego la aplicamos a los datos de prueba es muy común en el machine learning. Por lo tanto, scikit-learn tiene un atajo para esto `pipeline`:

In [5]:
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())
pipeline.fit(texto_entrenamiento, y_entrenamiento)
pipeline.score(texto_prueba, y_prueba)

0.9641319942611191

El código es mucho más corto y fácil de manejar. La construcción de piepelines no solo simplifica el código, sino que también es importante para la selección del modelo. Si queremos hacer una búsqueda grid C para ajustar al modelo de Regresión logística, hacemos: 

In [6]:
from sklearn.model_selection import GridSearchCV

vectorizer = TfidfVectorizer()
vectorizer.fit(texto_entrenamiento)

X_entrenamiento = vectorizer.transform(texto_entrenamiento)
X_prueba = vectorizer.transform(texto_prueba)

clf = LogisticRegression()
grid = GridSearchCV(clf, param_grid={'C': [.1, 1, 10, 100]}, cv=5)
grid.fit(X_entrenamiento, y_entrenamiento)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

1 . Puedes encontrar el error que se encuentra aquí. Esto se denomina "contaminación" del conjunto de pruebas y conduce a estimaciones demasiado optimistas del rendimiento de  generalización, o parámetros mal seleccionados.

In [7]:
# Tu respuesta

Con pipeline también se puede buscar parámetros de extracción de características con `GridSearchCV`:

In [8]:
from sklearn.model_selection import GridSearchCV

pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())

params = {'logisticregression__C': [.1, 1, 10, 100],
          "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (2, 2)]}
grid = GridSearchCV(pipeline, param_grid=params, cv=5)
grid.fit(texto_entrenamiento, y_entrenamiento)
print(grid.best_params_)
grid.score(texto_prueba, y_prueba)

{'logisticregression__C': 100, 'tfidfvectorizer__ngram_range': (1, 2)}


0.9863701578192252

2 .Cree un pipeline a una regresión  Ridge y `StandardScaler` en el conjunto de datos de Boston ( `sklearn.datasets.load_boston`). Agrega el transformador `sklearn.preprocessing.PolynomialFeatures` como segundo paso de preprocesamiento y una búsqueda grid del grado de los polinomios (prueba `1`, `2` y `3`).


In [9]:
# Tu solucion